In [1]:
# imports
from pickle import TRUE
import pandas as pd
import requests
import time
import warnings
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import base64
from io import StringIO
import creds


warnings.filterwarnings("ignore")

In [2]:
service=Service(ChromeDriverManager().install())

# Instanciando o Objeto ChromeOptions
options = webdriver.ChromeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Chrome(service=service, options=options)


In [3]:
wd_Chrome.get("https://www.fundamentus.com.br/")

wd_Chrome.find_element('xpath', '/html/body/div[1]/div[1]/div[1]/span/a[1]').click()


wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/form/input').click()


In [4]:
table_element = wd_Chrome.find_element('id', 'resultado')

# Get the HTML content of the table
table_html = table_element.get_attribute('outerHTML')

# Use Pandas to read the HTML and create a DataFrame
df = pd.read_html(table_html)[0]

In [5]:
lista_papel = df.Papel.to_list()

In [8]:
lista_empresa = []
lista_setor = []
lista_subsetor = []
lista_mes = []
lista_30d = []
lista_12m = []

for papel in lista_papel:
    wd_Chrome.get("https://www.fundamentus.com.br/")

    wd_Chrome.find_element('xpath', '/html/body/div[1]/div[1]/div[1]/span/a[1]').click()


    wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/form/input').click()

    # clica na ação
    wd_Chrome.find_element('xpath', f'//a[@href="detalhes.php?papel={papel}"]').click()

    # trazer empresa e o setor
    empresa = wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/table[1]/tbody/tr[3]/td[2]/span').text
    setor = wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/table[1]/tbody/tr[4]/td[2]/span/a').text
    subsetor = wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/table[1]/tbody/tr[5]/td[2]/span/a').text

    mes = wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/table[3]/tbody/tr[3]/td[2]/span/font').text
    _30d = wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/table[3]/tbody/tr[4]/td[2]/span/font').text
    _12m = wd_Chrome.find_element('xpath', '/html/body/div[1]/div[2]/table[3]/tbody/tr[5]/td[2]/span/font').text


    time.sleep(1.5)

    lista_empresa.append(empresa)
    lista_setor.append(setor)
    lista_subsetor.append(subsetor)
    lista_mes.append(mes)
    lista_30d.append(_30d)
    lista_12m.append(_12m)

    print(empresa)

PRINER ON NM
ATOMPAR ON
WDC NETWORKS ON NM
LOCAWEB ON NM
EMBRAER ON
CIA MARCOPOLO PN
CIA MARCOPOLO ON
3R PETROLEUM ON NM
EXCELSIOR PN
AMPLA ENERG ON
MOVIDA ON NM
COSAN SA INDUSTRIA E COMERCIO ON
OUROFINO S/A ON NM
DOHLER ON
DOHLER PN
USIMINAS PNB N1
MAGAZ LUIZA ON NM
MINUPAR ON
HAPVIDA ON NM
BAUMER ON
QUERO-QUERO ON NM
BAUMER PN
RECRUSUL ON
CSN ON
CLEARSALE ON NM
ALPARGATAS ON N1
PORTOBELLO S/A ON
USIMINAS PNA N1
QUALICORP ON NM
USIMINAS ON N1
ALPARGATAS PN N1
COGNA ON
AMBIPAR ON NM
TELEBRAS ON
MRV ON NM
ORIZON ON NM
ESPACOLASER ON NM
ANIMA ON NM
IOCHPE-MAXION ON
GUARARAPES CONFECÇÕES ON
OCEANPACT ON NM
SYN PROP TEC ON NM
MELIUZ ON NM
TELEBRAS PN
NORDON INDS METALURGICAS SA ON
ZAMP S.A. ON
Metalfrio ON NM
AERIS ON NM
ESTAPAR ON NM
ENJOEI ON NM
DIAGNOSTICOS DA AMERICA SA ON
AZEVEDO & TRAVASSOS ON
RECRUSUL PN
AZEVEDO & TRAVASSOS PN
CBA ON NM
BIOMM SA ON
BRF Foods ON NM
SER EDUCA ON NM
B TECH EQI ON
Tenda ON NM
KORA SAUDE ON NM
CEEE-D ON N1
INFRACOMM ON NM
ADVANCED-DH ON
AZUL PN
ALLIAR ON

In [9]:
df['empresa'] = lista_empresa
df['setor'] = lista_setor
df['sub_setor'] = lista_subsetor
df['var_mes'] = lista_mes
df['var_30d'] = lista_30d
df['var_12m'] = lista_12m



In [10]:

# Function to push DataFrame to GitHub
def push_dataframe_to_github(dataframe, file_name, repository):
    # Convert DataFrame to CSV in-memory
    csv_data = StringIO()
    dataframe.to_csv(csv_data, index=False)
    csv_content = csv_data.getvalue()

    # Encode content to Base64
    encoded_content = base64.b64encode(csv_content.encode()).decode()

    # Check if the file exists
    url_get = f'https://api.github.com/repos/{username}/{repository}/contents/{file_name}'
    headers_get = {'Authorization': f'token {creds.token}'}

    response_get = requests.get(url_get, headers=headers_get)
    response_get_json = response_get.json()

    # Extract the SHA from the response
    current_sha = response_get_json.get('sha')

    # Push data directly to GitHub using GitHub API
    url_put = f'https://api.github.com/repos/{username}/{repository}/contents/{file_name}'
    headers_put = {
        'Authorization': f'token {creds.token}',
        'Content-Type': 'application/json'
    }

    data_put = {
        'message': f'Update {file_name}',
        'content': encoded_content
    }

    # If the file exists, update it; otherwise, create it
    if current_sha is not None:
        data_put['sha'] = current_sha

    response_put = requests.put(url_put, headers=headers_put, json=data_put)

    if response_put.status_code == 200:
        print(f'Data pushed to {file_name} on GitHub successfully!')
    elif response_put.status_code == 201:
        print(f'{file_name} created on GitHub successfully!')
    else:
        print(f'Error: {response_put.status_code}, {response_put.text}')

# GitHub repository details
username = 'alanhassan'
repository = 'stocks_panel'

In [2]:
df = pd.read_csv('df.csv')

In [3]:
df.head()

,Papel,empresa,setor,sub_setor,Cotação,P/L,EV/EBITDA,P/VP,ROE,Div.Yield,Cresc. Rec.5a,Dív.Brut/ Patrim.,var_mes,var_30d,var_12m
0,PRNR3,PRINER ON NM,Serviços Diversos,Serviços Diversos,11.65,-1672.61,4.98,2.00,-0.12,0.00,36.30,1.20,-0.43,2.92,40.87
1,ATOM3,ATOMPAR ON,Outros,Outros,2.14,-628.92,-11.12,2.78,-0.44,14.97,20.53,0.00,0.47,0.94,-0.37
2,LVTC3,WDC NETWORKS ON NM,Programas e Serviços,Programas e Serviços,4.08,-499.67,0.41,0.41,-0.08,0.33,-10.31,0.00,-2.63,-0.24,-51.14
3,LWSA3,LOCAWEB ON NM,Programas e Serviços,Programas e Serviços,6.13,-412.38,14.16,1.25,-0.30,0.85,41.70,0.02,2.00,2.51,-9.99
4,EMBR3,EMBRAER ON,Material de Transporte,Material Aeronáutico e de Defesa,21.90,-397.05,6.68,1.27,-0.32,0.00,21.60,1.12,-2.19,-5.36,45.13


In [4]:
# filtering columns

df = df[['Papel', 'empresa', 'setor', 'sub_setor', 'Cotação', 'P/L', 'EV/EBITDA', 'P/VP', 'ROE', 'Div.Yield', 'Cresc. Rec.5a', 'Dív.Brut/ Patrim.', 'var_mes', 'var_30d', 'var_12m']]

In [5]:
df.head()

,Papel,empresa,setor,sub_setor,Cotação,P/L,EV/EBITDA,P/VP,ROE,Div.Yield,Cresc. Rec.5a,Dív.Brut/ Patrim.,var_mes,var_30d,var_12m
0,PRNR3,PRINER ON NM,Serviços Diversos,Serviços Diversos,11.65,-1672.61,4.98,2.00,-0.12,0.00,36.30,1.20,-0.43,2.92,40.87
1,ATOM3,ATOMPAR ON,Outros,Outros,2.14,-628.92,-11.12,2.78,-0.44,14.97,20.53,0.00,0.47,0.94,-0.37
2,LVTC3,WDC NETWORKS ON NM,Programas e Serviços,Programas e Serviços,4.08,-499.67,0.41,0.41,-0.08,0.33,-10.31,0.00,-2.63,-0.24,-51.14
3,LWSA3,LOCAWEB ON NM,Programas e Serviços,Programas e Serviços,6.13,-412.38,14.16,1.25,-0.30,0.85,41.70,0.02,2.00,2.51,-9.99
4,EMBR3,EMBRAER ON,Material de Transporte,Material Aeronáutico e de Defesa,21.90,-397.05,6.68,1.27,-0.32,0.00,21.60,1.12,-2.19,-5.36,45.13


In [6]:
df.dtypes

Papel                 object
empresa               object
setor                 object
sub_setor             object
Cotação              float64
P/L                  float64
EV/EBITDA            float64
P/VP                 float64
ROE                  float64
Div.Yield            float64
Cresc. Rec.5a        float64
Dív.Brut/ Patrim.    float64
var_mes              float64
var_30d              float64
var_12m              float64
dtype: object

In [7]:
# adjusting types

df['P/L'] = df['P/L'].str.replace('[.,]', '', regex=True).astype(int)
df['EV/EBITDA'] = df['EV/EBITDA'].str.replace('[.,]', '', regex=True).astype(int)
df['P/VP'] = df['P/VP'].str.replace('[.,]', '', regex=True).astype(int)
df['ROE'] = df['ROE'].str.replace('[.,%]', '', regex=True).astype(int)
df['Div.Yield'] = df['Div.Yield'].str.replace('[.,%]', '', regex=True).astype(int)
df['Cresc. Rec.5a'] = df['Cresc. Rec.5a'].str.replace('[.,%]', '', regex=True).astype(int)
df['var_mes'] = df['var_mes'].str.replace('[.,%]', '', regex=True).astype(int)
df['var_30d'] = df['var_30d'].str.replace('[.,%]', '', regex=True).astype(int)
df['var_12m'] = df['var_12m'].str.replace('[.,%]', '', regex=True).astype(int)

AttributeError: Can only use .str accessor with string values!

In [ ]:
# adjusting columns

df['Cotação'] = df['Cotação']/100
df['P/L'] = df['P/L']/100
df['EV/EBITDA'] = df['EV/EBITDA']/100
df['P/VP'] = df['P/VP']/100
df['ROE'] = df['ROE']/100
df['Div.Yield'] = df['Div.Yield']/100
df['Dív.Brut/ Patrim.'] = df['Dív.Brut/ Patrim.']/100
df['var_mes'] = df['var_mes']/100
df['var_30d'] = df['var_30d']/100
df['var_12m'] = df['var_12m']/100
df['Cresc. Rec.5a'] = df['Cresc. Rec.5a']/100

In [8]:
df.head()

,Papel,empresa,setor,sub_setor,Cotação,P/L,EV/EBITDA,P/VP,ROE,Div.Yield,Cresc. Rec.5a,Dív.Brut/ Patrim.,var_mes,var_30d,var_12m
0,PRNR3,PRINER ON NM,Serviços Diversos,Serviços Diversos,11.65,-1672.61,4.98,2.00,-0.12,0.00,36.30,1.20,-0.43,2.92,40.87
1,ATOM3,ATOMPAR ON,Outros,Outros,2.14,-628.92,-11.12,2.78,-0.44,14.97,20.53,0.00,0.47,0.94,-0.37
2,LVTC3,WDC NETWORKS ON NM,Programas e Serviços,Programas e Serviços,4.08,-499.67,0.41,0.41,-0.08,0.33,-10.31,0.00,-2.63,-0.24,-51.14
3,LWSA3,LOCAWEB ON NM,Programas e Serviços,Programas e Serviços,6.13,-412.38,14.16,1.25,-0.30,0.85,41.70,0.02,2.00,2.51,-9.99
4,EMBR3,EMBRAER ON,Material de Transporte,Material Aeronáutico e de Defesa,21.90,-397.05,6.68,1.27,-0.32,0.00,21.60,1.12,-2.19,-5.36,45.13


In [16]:
df['prov'] = df['Papel'].str[:4]

In [20]:
df.drop_duplicates(subset='prov', keep='first', inplace=True)

In [23]:
df.drop(columns=['prov'], inplace=True)

In [24]:
df.shape

(339, 15)

In [27]:
# Push each DataFrame to GitHub
push_dataframe_to_github(df, 'df.csv', repository)

Data pushed to df.csv on GitHub successfully!


In [26]:
df.head()

,Papel,empresa,setor,sub_setor,Cotação,P/L,EV/EBITDA,P/VP,ROE,Div.Yield,Cresc. Rec.5a,Dív.Brut/ Patrim.,var_mes,var_30d,var_12m
0,PRNR3,PRINER ON NM,Serviços Diversos,Serviços Diversos,11.65,-1672.61,4.98,2.00,-0.12,0.00,36.30,1.20,-0.43,2.92,40.87
1,ATOM3,ATOMPAR ON,Outros,Outros,2.14,-628.92,-11.12,2.78,-0.44,14.97,20.53,0.00,0.47,0.94,-0.37
2,LVTC3,WDC NETWORKS ON NM,Programas e Serviços,Programas e Serviços,4.08,-499.67,0.41,0.41,-0.08,0.33,-10.31,0.00,-2.63,-0.24,-51.14
3,LWSA3,LOCAWEB ON NM,Programas e Serviços,Programas e Serviços,6.13,-412.38,14.16,1.25,-0.30,0.85,41.70,0.02,2.00,2.51,-9.99
4,EMBR3,EMBRAER ON,Material de Transporte,Material Aeronáutico e de Defesa,21.90,-397.05,6.68,1.27,-0.32,0.00,21.60,1.12,-2.19,-5.36,45.13
